In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_google_genai langchain_community langchain_core tavily-python wikipedia

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is not set in the .env file.")
print("Gemini api key loaded successfully")

 

Gemini api key loaded successfully


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI 
llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key =GEMINI_API_KEY)

In [ ]:
from typing import List,Optional
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Image, display

llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model ="gemini-1.5-flash",
                                                      api_key = GEMINI_API_KEY)

class Company(BaseModel):
    company_name: str = Field(description="Suggested Name of the company")
    employees: int = Field(description="Estimated number of employees in the company")
    description: str = Field(description="Short summary of the company")
    location:str = Field(description="Random Location of the company")

class CompanyList(BaseModel):
    companies: List[Company] = Field(description="List of companies")

class GenerateCompaniesState(TypedDict):
    topic: str
    max_companies: int
    companies: List[Company]

company_instructions = """You are a business developer known for identifying cutting-edge
                           opportunities.Your task is to create {max_companies} innovative
                           companies related to the future of {company_name}. For each
                           company, craft a company_name, a brief description highlighting
                           its innovative aspect, expected employee size, and target location."""

def create_companies(state: GenerateCompaniesState):
    company_name = state['topic']
    max_companies = state['max_companies']
    structured_llm = llm.with_structured_output(CompanyList)
    system_message = company_instructions.format(company_name=company_name, max_companies=max_companies)
    companies = structured_llm.invoke([SystemMessage(content=system_message)] + [HumanMessage(content="Generate the list of companies.")])
    return {'companies':companies.companies}



building_graph = StateGraph(GenerateCompaniesState)
building_graph.add_node('create_companies', create_companies)
building_graph.add_edge(START,'create_companies')
building_graph.add_edge('create_companies',END)
compiled_graph = building_graph.compile()
display(Image(compiled_graph.get_graph().draw_mermaid_png()))
state = {"topic": "fast food","max_companies": 3}
x=compiled_graph.invoke(state)
print(x)


In [ ]:
from typing import List,Optional
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Image, display

llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model ="gemini-1.5-flash",
                                                      api_key = GEMINI_API_KEY)

class Company(BaseModel):
    company_name: str = Field(description="Suggested Name of the company")
    employees: int = Field(description="Estimated number of employees in the company")
    description: str = Field(description="Short summary of the company")
    location:str = Field(description="Random Location of the company")

class CompanyList(BaseModel):
    companies: List[Company] = Field(description="List of companies")

class GenerateCompaniesState(TypedDict):
    topic: str
    max_companies: int
    companies: List[Company]

company_instructions = """You are helpfull business developer. You task is to suggest {max_companies} business names along with number of suggested employess and some summary and some random location in pakistan for domain: {company_name}"""

def create_companies(state: GenerateCompaniesState):
    company_name = state['topic']
    max_companies = state['max_companies']
    structured_llm = llm.with_structured_output(CompanyList)
    system_message = company_instructions.format(company_name=company_name, max_companies=max_companies)
    companies = structured_llm.invoke([HumanMessage(content=system_message)] )
    return {'companies':companies.companies}



building_graph = StateGraph(GenerateCompaniesState)
building_graph.add_node('create_companies', create_companies)
building_graph.add_edge(START,'create_companies')
building_graph.add_edge('create_companies',END)
compiled_graph = building_graph.compile()
display(Image(compiled_graph.get_graph().draw_mermaid_png()))
state = {"topic": "fast food","max_companies": 3}
x=compiled_graph.invoke(state)
print(x)


In [ ]:
from typing import List,Optional
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage
from typing_extensions import TypedDict
from IPython.display import Image, display

llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model ="gemini-1.5-flash",
                                                      api_key = GEMINI_API_KEY)
class Company(BaseModel):
    name: str = Field(description="Name of the company")
    employees: int = Field(description="Total number of employees in the company")
    description: str = Field(description="Description of the company")
    location:str = Field(description="Location of the company")

class CompanyList(BaseModel):
    companies: List[Company] = Field(description="List of companies")

class GenerateCompaniesState(TypedDict):
    topic: str
    max_companies: int
    companies: List[Company]

company_instructions = """You are a business developer known for identifying cutting-edge opportunities.
Your task is to create {max_companies} innovative companies related to the future of {company_name}.
For each company, craft a name, a brief description highlighting its innovative aspect, expected employee size, and target location.
For each company, provide the following information in JSON format:
```json
{{
  "name": "Company Name (string)",
  "employees": "Number of employees (integer)",
  "description": "Brief description of the company and its innovative aspect (string)",
  "location": "Target location of the company (string)"
}}

```"""

def create_companies(state: GenerateCompaniesState):


    company_name = state['topic']
    max_companies = state['max_companies']

    structured_llm = llm.with_structured_output(CompanyList)

    system_message = company_instructions.format(company_name=company_name, max_companies=max_companies)

    companies = llm.invoke([SystemMessage(content=system_message)] + [HumanMessage(content="Generate the list of companies.")])

    return companies

state = {"topic": "cars","max_companies": 6}
companies = create_companies(state)
print(companies)


In [ ]:
from typing import List,Optional
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from IPython.display import Image, display

llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model ="gemini-1.5-flash",
                                                      api_key = GEMINI_API_KEY)
class Company(BaseModel):
    company_name: str = Field(description="Name of the company")
    employees: int = Field(description="Total number of employees in the company")
    description: str = Field(description="Description of the company")
    location:str = Field(description="Location of the company")

class CompanyList(BaseModel):
    companies: List[Company] = Field(description="List of companies")

class GenerateCompaniesState(TypedDict):
    topic: str
    max_companies: int
    companies: List[Company]


company_instructions = """You are tasked with creating a set of new companies. Follow these instructions carefully:

1. Review the topic:
{topic}

2. Generate a list of {max_companies} companies related to the topic.

3. For each company, provide a name of the company.

4. For each company, provide a description of the company.

5. For each company, provide a execpected number of employees.

5. For each company, provide a location of the company .
"""


def create_companies(state: GenerateCompaniesState):
    """Create companies based on the topic."""

    topic = state['topic']
    max_companies = state['max_companies']

    structured_llm = llm.with_structured_output(CompanyList)

    system_message = company_instructions.format(topic=topic, max_companies=max_companies)

    companies = structured_llm.invoke([SystemMessage(content=system_message)] + [HumanMessage(content="Generate the list of companies.")])

    return {'companies':companies.companies}

building_graph = StateGraph(GenerateCompaniesState)
building_graph.add_node('create_companies', create_companies)
building_graph.add_edge(START,'create_companies')
building_graph.add_edge('create_companies',END)
compiled_graph = building_graph.compile()
display(Image(compiled_graph.get_graph().draw_mermaid_png()))
state = {"topic": "cars","max_companies": 3}
x=compiled_graph.invoke(state)
print(x)

In [ ]:
from typing import List,Optional
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage
from typing_extensions import TypedDict
from IPython.display import display, Image
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph,START,END
from IPython.display import Image, display

llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model ="gemini-1.5-flash",
                                                      api_key = GEMINI_API_KEY)
class Company(BaseModel):
    company_name: str = Field(description="Name of the company")
    employees: int = Field(description="Total number of employees in the company")
    description: str = Field(description="Description of the company")
    location:str = Field(description="Location of the company")

class CompanyList(BaseModel):
    companies: List[Company] = Field(description="List of companies")

class GenerateCompaniesState(TypedDict):
    topic: str
    max_companies: int
    companies: List[Company]
 
llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key ='GEMINI_API_KEY')

company_instructions = """You are a business developer known for identifying cutting-edge opportunities.
Your task is to create {max_companies} innovative companies related to the future of {company_name}.
For each company, craft a company_name, a brief description highlighting its innovative aspect, expected employee size, and target location.
"""

def create_companies(state: GenerateCompaniesState):


    company_name = state['topic']
    max_companies = state['max_companies']

    structured_llm = llm.with_structured_output(CompanyList)

    system_message = company_instructions.format(company_name=company_name, max_companies=max_companies)

    companies = structured_llm.invoke([SystemMessage(content=system_message)] + [HumanMessage(content="Generate the list of companies.")])
   
    return {'companies':companies.companies}


building_graph = StateGraph(GenerateCompaniesState)
building_graph.add_node('create_companies', create_companies)
building_graph.add_edge(START,'create_companies')
building_graph.add_edge('create_companies',END)
compiled_graph = building_graph.compile()
display(Image(compiled_graph.get_graph().draw_mermaid_png()))
state = {"topic": "cars","max_companies": 3}
x=compiled_graph.invoke(state)
print(x)